# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [69]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [70]:
csv_path = 'WeatherPy.csv'
vacation_data = pd.read_csv(csv_path)
vacation_data.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Khatanga,71.9667,102.5000,59.52,59,72,8.48,RU,1659929088
1,Avera,33.1940,-82.5271,77.05,78,96,4.21,US,1659929089
2,Cherskiy,68.7500,161.3000,52.03,48,44,5.35,RU,1659929089
3,New Richmond,45.1230,-92.5366,65.10,96,100,8.05,US,1659929090
4,Calama,-22.4667,-68.9333,45.77,20,0,5.97,CL,1659929090


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [71]:
#showing max humidity for heatmap
maxhumidity = vacation_data['Humidity'].astype(float).max()
vacation_data['Humidity'].astype(float)

0       59.0
1       78.0
2       48.0
3       96.0
4       20.0
       ...  
567     21.0
568     61.0
569     98.0
570     20.0
571    100.0
Name: Humidity, Length: 572, dtype: float64

In [72]:
#running a count to ensure all rows are equal in count
vacation_data.count()

City          572
Lat           572
Long          572
Max Temp      572
Humidity      572
Cloudiness    572
Wind Speed    572
Country       570
Date          572
dtype: int64

In [73]:
#saving gmaps figure as vacation figure
vacation_fig = gmaps.figure()

#configure gmaps
gmaps.configure(api_key=g_key)


#Using Lat and Long to make location for api call and saving them as float
locations = vacation_data[['Lat', 'Long']].astype(float)

#converting humidity to a float for the call
humidityrate = vacation_data['Humidity'].astype(float)
maxhumidity = vacation_data['Humidity'].astype(float).max()

#now I create the heat map!
heatmap = gmaps.heatmap_layer(locations, weights=humidityrate,dissipating=False, max_intensity=maxhumidity,point_radius=3)

#adding layer to figure
vacation_fig.add_layer(heatmap)

#showing figure
vacation_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Narrow down the DataFrame to find your ideal weather condition. For example:


A max temperature lower than 80 degrees but higher than 70.


Wind speed less than 10 mph.


Zero cloudiness.


Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.

In [76]:
#making a new data frame per instructions, which I put above. I also added a loc for humidity. 
ideal_weather = vacation_data.loc[(vacation_data["Max Temp"] > 70) & (vacation_data["Max Temp"] < 80) & (vacation_data["Cloudiness"] == 0) & (vacation_data["Wind Speed"] < 10) & (vacation_data['Humidity'] < 50), :]
ideal_weather = ideal_weather.dropna()
ideal_weather.reset_index(inplace=True)
ideal_weather

,index,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,56,Puertollano,38.6871,-4.1073,75.07,39,0,3.56,ES,1659929112
1,69,Tiznit Province,29.5833,-9.5000,73.62,40,0,3.69,MA,1659929116
2,77,Sítio Novo do Tocantins,-5.5183,-47.4611,76.48,43,0,3.49,BR,1659929120
3,162,Miles City,46.4083,-105.8406,74.37,40,0,5.75,US,1659929153
4,336,Semey,50.4111,80.2275,73.62,43,0,6.71,KZ,1659929219
5,339,Fasa,28.9383,53.6482,72.64,49,0,4.61,IR,1659929220
6,343,Qūchān,37.1060,58.5095,73.02,36,0,5.44,IR,1659929223
7,518,Aksarka,66.5606,67.7975,76.75,49,0,9.66,RU,1659929295


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [98]:
#setting up list/dictionary to store response information from the api call
hotel_list = []  

#creating a loop for the api call 

count=1 

print('Beginning Data Retrieval')     
print('---------------------------')

for i in range(len(ideal_weather)):
    query_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #locating latitudes & longitudes
    lat = ideal_weather.loc[i]['Lat']    
    lon = ideal_weather.loc[i]['Long'] 
    
    #Print statement as starter code requested in Weather notebook
    print('Looking for a hotel nearby ' + str(ideal_weather.loc[i]['City']))
    
    #setting up parameters
    parameters = {"location": f"{lat},{lon}",
        "radius": 5000,
        "keyword" : "hotel",
        "key": g_key}
    
    #run api request
    hotel_response = requests.get(base_url, params=parameters)
    hotel_data = hotel_response.json()
    
    try:
        #storing necessary response data as a variable
        hotel_name = hotel_data['results'][0]['name']
        hotel_lat = hotel_data['results'][0]['geometry']['location']['lat']
        hotel_lon = hotel_data['results'][0]['geometry']['location']['lng']
        hotel_city = ideal_weather.loc[i]["City"]
        hotel_country = ideal_weather.loc[i]["Country"]
        
        #appending hotel list
        hotel_list.append({"City": hotel_city,
                           "Country": hotel_country,
                           "Hotel Name": hotel_name,
                           "Lat": hotel_lat,
                           "Lng": hotel_lon})
    except:
        print("Hotel not found. Skipping...")
        
    count=count+1
    
    pass

print('---------------------------')
print('Data Retrieval Complete')
print('---------------------------')

Beginning Data Retrieval
---------------------------
Looking for a hotel nearby Puertollano
Looking for a hotel nearby Tiznit Province
Hotel not found. Skipping...
Looking for a hotel nearby Sítio Novo do Tocantins
Looking for a hotel nearby Miles City
Looking for a hotel nearby Semey
Looking for a hotel nearby Fasa
Looking for a hotel nearby Qūchān
Looking for a hotel nearby Aksarka
Hotel not found. Skipping...
---------------------------
Data Retrieval Complete
---------------------------


In [99]:
#converting dictionary into dataframe
hotel_df = pd.DataFrame(hotel_list)
hotel_df

,City,Country,Hotel Name,Lat,Lng
0,Puertollano,ES,Hotel Santa Eulalia/ Restaurante La Ula,38.687843,-4.106362
1,Sítio Novo do Tocantins,BR,Imperial Hotel,-5.528473,-47.470696
2,Miles City,US,Sleep Inn & Suites,46.398615,-105.821787
3,Semey,KZ,Grand Hotel Semey,50.418826,80.258959
4,Fasa,IR,Jahangardi Hotel,28.943129,53.639358
5,Qūchān,IR,هتل اپارتمان سلطانی,37.089208,58.515353


In [100]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [102]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
vacation_fig = gmaps.figure()
vacation_fig.add_layer(heatmap)
vacation_fig.add_layer(markers)

vacation_fig

Figure(layout=FigureLayout(height='420px'))